#Preliminaries

Run all of them at the start

In [2]:
from collections import defaultdict
from itertools import product
from sympy import (
    symbols, Rational, sqrt, pi, I, exp, cos, sin,
    Abs, Mod, simplify, conjugate, re, im, Matrix, zeros
)

In [12]:
def add_kets(*kets):
    """
    Adds multiple pure states (kets) by summing coefficients of identical basis states.
    """
    coeff_dict = defaultdict(lambda: 0)

    for ket in kets:
        for coeff, basis in ket:
            coeff_dict[basis] += coeff

    return sorted([(coeff, basis) for basis, coeff in coeff_dict.items() if coeff != 0], key=lambda x: x[1])

def merge_mixed_states(mixed_state):
    """
    Merges identical pure states in a mixed state and removes states with zero probability.

    Uses sorting only on basis (not coefficients) to ensure symbolic compatibility.
    """
    merged_state = {}

    for prob, state in mixed_state:
        # Sort by basis only (not coeffs) to avoid symbolic comparison issues
        state_key = tuple(sorted(state, key=lambda x: x[1]))

        if state_key in merged_state:
            merged_state[state_key] += prob
        else:
            merged_state[state_key] = prob

    return [(prob, list(state_key)) for state_key, prob in merged_state.items() if prob != 0]

def tensor_product(state1, state2):
    """
    Computes the tensor product of two pure quantum states.

    Parameters:
    - state1: A list of (coefficient, basis) tuples representing the first pure state.
    - state2: A list of (coefficient, basis) tuples representing the second pure state.

    Returns:
    - A new pure state representing the tensor product of the two input states.
    """
    return [
        (c1 * c2, b1 + b2)  # Multiply coefficients, concatenate basis tuples
        for c1, b1 in state1
        for c2, b2 in state2
    ]


def comp_st(*states):
    """
    Computes the tensor product of multiple pure quantum states.

    Parameters:
    - *states: Any number of pure states (each a list of (coefficient, basis) tuples).

    Returns:
    - A single pure state representing the full tensor product.
      If no states are given, returns the identity: [(1, ())].
    """
    if not states:
        return [(1, ())]  # Identity for tensor product

    result = states[0]
    for state in states[1:]:
        result = tensor_product(result, state)
    return result


def comp_mixed_states(*mixed_states):
    """
    Computes the tensor product of multiple mixed quantum states.

    Parameters:
    - *mixed_states: Any number of mixed states.
      Each is a list of (probability, pure_state) tuples.

    Returns:
    - A new mixed state as a list of (probability_product, tensor_product_pure_state),
      with identical pure states merged and zero-probability states removed.
    """
    if not mixed_states:
        return []

    result = []

    # Iterate over all combinations (Cartesian product) of component states
    for combo in product(*mixed_states):
        prob = 1
        state = [(1, ())]  # Start with identity state

        # Compute tensor product and combined probability
        for p, s in combo:
            prob *= p
            state = tensor_product(state, s)

        result.append((prob, state))

    return merge_mixed_states(result)

def create_zero_state(N):
    """
    Generates an N-qubit |0> state.

    Returns a pure state representation.
    """
    return [(1, tuple(0 for _ in range(N)))]

def create_bell_state(dim, shift, phase):
    """
    Generates a generalized Bell state:

    sum_{k=0}^{d-1} 1/sqrt(d) * omega^(k * phase) * |k> ⊗ |k+shift>

    where omega = exp(2 * pi * i / d)
    """
    omega = exp(2 * pi * I / dim)  # Root of unity
    normalization = 1 / sqrt(dim)

    return [(normalization * omega**(k * phase), (k, Mod(k + shift, dim))) for k in range(dim)]

def pure_to_mixed(pure_state):
    """
    Converts a pure state into a mixed state with a single entry having probability 1.

    Returns a mixed state representation.
    """
    return [(1, pure_state)]

def pauli_x(n, state):
    """Applies the Pauli-X gate to qubit n."""
    return [(coeff, tuple(v[:n] + (1 - v[n],) + v[n+1:])) for coeff, v in state]

def pauli_z(n, state):
    """Applies the Pauli-Z gate to qubit n."""
    return [((-coeff if v[n] == 1 else coeff), v) for coeff, v in state]

def pauli_y(n, state):
    """Applies the Pauli-Y gate to qubit n."""
    return [(I * coeff, tuple(v[:n] + (1 - v[n],) + v[n+1:])) for coeff, v in state if v[n] == 0] + \
           [(-I * coeff, tuple(v[:n] + (1 - v[n],) + v[n+1:])) for coeff, v in state if v[n] == 1]

def hadamard(n, state):
    """
    Applies the Hadamard gate to qubit n.
    If the n-th qubit is 0, it transforms into (|0> + |1>) / sqrt(2).
    If the n-th qubit is 1, it transforms into (|0> - |1>) / sqrt(2).
    Other qubits remain unchanged.
    """
    sqrt_2_inv = 1/sqrt(2)
    new_state = []

    for coeff, basis in state:
        if basis[n] == 0:
            new_state.append((sqrt_2_inv * coeff, tuple(basis[:n] + (0,) + basis[n+1:])))
            new_state.append((sqrt_2_inv * coeff, tuple(basis[:n] + (1,) + basis[n+1:])))
        else:
            new_state.append((sqrt_2_inv * coeff, tuple(basis[:n] + (0,) + basis[n+1:])))
            new_state.append((-sqrt_2_inv * coeff, tuple(basis[:n] + (1,) + basis[n+1:])))

    return add_kets(new_state)

def rotation_x(n, angle, state):
    """
    Applies the Rx (rotation around X-axis) gate to qubit n with given angle.

    Rx(angle) = cos(angle/2) * I - i * sin(angle/2) * X
    """
    cos_term = cos(angle / 2)
    sin_term = -I * sin(angle / 2)

    identity_part = [(cos_term * coeff, basis) for coeff, basis in state]
    x_part = [(sin_term * coeff, tuple(basis[:n] + (1 - basis[n],) + basis[n+1:])) for coeff, basis in state]

    return add_kets(identity_part + x_part)

def cnot(control, target, state):
    """
    Applies the CNOT gate with `control` and `target` qubits on `state`.
    """
    new_state = []

    for coeff, basis in state:
        ctrl_val = basis[control]
        tgt_val = basis[target]

        flipped = tgt_val if ctrl_val == 0 else 1 - tgt_val

        new_basis = basis[:target] + (flipped,) + basis[target+1:]
        new_state.append((coeff, new_basis))

    return new_state

def cz(control, target, state):
    """
    Applies the CZ (Controlled-Z) gate.
    If both control and target qubits are 1, the coefficient gets a -1 phase.
    """
    return [(coeff if not (basis[control] == 1 and basis[target] == 1) else -coeff, basis) for coeff, basis in state]

def id(state):
    return state

def apply_gate_to_mixed_state(gate_function, params, mixed_state):
    """
    Applies a quantum gate function to all pure states in a mixed state and merges identical states.

    Parameters:
    - gate_function: The function that applies a gate to a pure state.
    - params: A tuple of parameters required by the gate function (e.g., qubit index).
    - mixed_state: A list of (probability, pure_state) tuples.

    Returns:
    - A new mixed state with the gate applied and identical states merged.
    """
    new_mixed_state = [(prob, gate_function(*params, state)) for prob, state in mixed_state]

    # Use the predefined function to merge identical states
    return merge_mixed_states(new_mixed_state)

def scale_state(factor, state):
    """Multiplies each coefficient in a quantum state by `factor`."""
    return [(factor * coeff, basis) for coeff, basis in state]

def measurement(meas, indices, state):
    """
    Performs a measurement where `meas` is a bra, `state` is a ket, and `indices` specifies the measured qubits.

    - `meas` is a quantum state (bra) whose basis corresponds to a subset of qubits in `state`.
    - `indices` specifies which qubits in `state` correspond to `meas`.
    - The conjugate of `meas`'s coefficients is used.
    - If a basis in `meas` matches the respective indices in `state`, their coefficients are multiplied.
    - The measured qubits are traced out in the final output.
    """
    result = []

    for meas_coeff, meas_basis in meas:
        conj_meas_coeff = meas_coeff.conjugate()  # Conjugate coefficients in the bra

        for state_coeff, state_basis in state:
            # Extract relevant qubits from the state basis using indices
            extracted_basis = tuple(state_basis[i] for i in indices)

            # If the extracted basis matches the measurement basis, apply the measurement
            if extracted_basis == meas_basis:
                # Compute new coefficient
                new_coeff = conj_meas_coeff * state_coeff

                # Remove measured qubits from the state basis
                new_basis = tuple(state_basis[i] for i in range(len(state_basis)) if i not in indices)

                # Append to result
                result.append((new_coeff, new_basis))

    return add_kets(result)  # Merge duplicate basis states and sum coefficients

def mixed_measurement(mixed_meas, indices, mixed_state):
    """
    Applies a mixed-state measurement to a mixed quantum state.

    Parameters:
    - mixed_meas: list of (probability, pure_bra) tuples
    - indices: list of qubit indices being measured
    - mixed_state: list of (probability, pure_ket) tuples

    Returns:
    - A new mixed state (list of (probability, pure_state)) after measurement and normalization
    """
    result_mixed = []

    for prob_m, meas in mixed_meas:
        for prob_s, state in mixed_state:
            projected = measurement(meas, indices, state)

            if projected:
                projected_norm, normalized_state = normalize(projected)
                if projected_norm != 0:
                    new_prob = prob_m * prob_s * projected_norm
                    result_mixed.append((new_prob, normalized_state))

    return merge_mixed_states(result_mixed)

def add_operators(*ops):
    """
    Adds multiple operators (each in mixed state form) and merges identical pure states.

    Parameters:
    - *ops: any number of mixed states (operators), each as a list of (probability, pure_state) tuples

    Returns:
    - A single mixed state with summed weights and merged duplicates.
    """
    combined = []
    for op in ops:
        combined.extend(op)
    return merge_mixed_states(combined)

def normalize(state):
    """
    Normalizes a quantum state and returns both the normalized state and the squared norm.

    Returns:
    - (norm, normalized_state)
    """
    norm = sum(abs(coeff)**2 for coeff, _ in state)

    if norm == 0:
        return 0, state  # Zero state, no normalization

    norm_sqrt = sqrt(norm)

    # Remove global phase
    for coeff, _ in state:
        if coeff != 0:
            phase_factor = coeff / abs(coeff)
            break
    else:
        phase_factor = 1

    normalized_state = scale_state(1 / norm_sqrt * conjugate(phase_factor), state)
    return norm, normalized_state

def normalized_mixed_state(mixed_state):
    """
    Normalizes a mixed quantum state and returns both the total probability (trace)
    and the normalized mixed state.

    Parameters:
    - mixed_state: list of (probability, pure_state) tuples

    Returns:
    - (total_probability, normalized_mixed_state)

    Notes:
    - If the total probability is zero, the original state is returned unnormalized.
    """
    total_prob = sum(prob for prob, _ in mixed_state)

    if total_prob == 0:
        return 0, mixed_state  # Can't normalize a zero-probability state

    normalized = [(prob / total_prob, state) for prob, state in mixed_state]
    return total_prob, normalized

def simplify_mixed_state(mixed_state):
    """
    Applies sympy's `simplify()` to all probabilities in the mixed state.

    Parameters:
    - mixed_state: list of (probability, pure_state) tuples

    Returns:
    - A new mixed state with simplified probabilities
    """
    return [(simplify(prob), state) for prob, state in mixed_state]

def fidelity(pure_state, mixed_state):
    """
    Computes the fidelity F(ρ, |ψ⟩⟨ψ|) = ∑ p_i * |⟨ψ|ψ_i⟩|^2
    using squared norm of the projection result.
    """
    total_fidelity = 0

    for prob, mixed_component in mixed_state:
        projection = measurement(pure_state, list(range(len(pure_state[0][1]))), mixed_component)
        norm_squared = sum(abs(coeff)**2 for coeff, _ in projection)
        total_fidelity += norm_squared * prob

    return total_fidelity

def trace_out(mixed_state, indices):
    """
    Traces out qubits at `indices` from a mixed state by performing identity measurement
    and summing over outcomes.

    Parameters:
    - mixed_state: list of (probability, pure_state) tuples
    - indices: list of qubit indices to trace out

    Returns:
    - Reduced mixed state over remaining qubits
    """
    # Define 1-qubit identity measurement as a mixed state
    id_meas_1q = [(1, [(1, (0,))]), (1, [(1, (1,))])]

    # Create full identity measurement over all qubits to be traced out
    full_meas = comp_mixed_states(*([id_meas_1q] * len(indices)))

    # Perform measurement and trace out
    return mixed_measurement(full_meas, indices, mixed_state)

def display_state(state, label=None):
    """
    Normalizes, removes global phase, sorts by basis, and prints the state as a flat list.
    """
    _, normalized = normalize(state)
    sorted_state = sorted(normalized, key=lambda x: x[1])
    if label:
        print(f"{label}: {sorted_state}")
    else:
        print(sorted_state)

def display_mixed_state(mixed_state, label=None):
    """
    Displays each (probability, pure_state) in a mixed state.
    - Respects SYMBOLIC_MODE (skips sorting by prob if symbolic).
    - Normalizes pure states and sorts them by basis.
    """
    if label:
        print(f"{label}:")

    if SYMBOLIC_MODE:
        states = mixed_state  # Don't sort by prob — may not be comparable
    else:
        states = sorted(mixed_state, key=lambda x: x[0], reverse=True)

    for i, (prob, pure_state) in enumerate(states):
        if SYMBOLIC_MODE:
            normalized = pure_state  # Don't normalize or simplify unless necessary
        else:
            _, normalized = normalize(pure_state)
            normalized = sorted(normalized, key=lambda x: x[1])

        print(f"  [{i}] Prob: {prob}, State: {normalized}")

def specify_bell_state(pure_state):
    """
    Compares a given pure state with known Bell states and returns the corresponding label.

    - "φ⁺" for create_bell_state(2, 0, 0)
    - "φ⁻" for create_bell_state(2, 0, 1)
    - "ψ⁺" for create_bell_state(2, 1, 0)
    - "ψ⁻" for create_bell_state(2, 1, 1)
    """
    bell_states = {
        tuple(create_bell_state(2, 0, 0)): "φ⁺",
        tuple(create_bell_state(2, 0, 1)): "φ⁻",
        tuple(create_bell_state(2, 1, 0)): "ψ⁺",
        tuple(create_bell_state(2, 1, 1)): "ψ⁻",
    }
    return bell_states.get(tuple(pure_state), "None")

def infer_num_qubits(pure_state):
    """
    Infers the number of qubits from the first basis in the pure state.

    Parameters:
    - pure_state: list of (coefficient, basis) tuples

    Returns:
    - Integer number of qubits
    """
    for _, basis in pure_state:
        return len(basis)
    return 0  # fallback for empty state

def mixed_to_DM(mixed_state):
    """
    Converts a mixed state into a full symbolic density matrix.
    The number of qubits is inferred automatically.
    """
    num_qubits = infer_num_qubits(mixed_state[0][1])
    dim = 2**num_qubits
    rho = zeros(dim, dim)

    def basis_index(basis):
        return int("".join(map(str, basis)), 2)

    for prob, state in mixed_state:
        for c1, b1 in state:
            i = basis_index(b1)
            for c2, b2 in state:
                j = basis_index(b2)
                rho[i, j] += prob * c1 * c2.conjugate()

    return simplify(rho)

def pauli_ev(obs, indices, state):
    """
    Computes ⟨ψ|O|ψ⟩ for a Pauli string operator O applied to a pure state.

    Parameters:
    - obs: a string like "XZ" or "yx" specifying the Pauli operators
    - indices: list of qubit indices the Pauli ops are applied to
    - state: a pure state (list of (coefficient, basis))

    Returns:
    - A symbolic expression for ⟨ψ|O|ψ⟩
    """
    if len(obs) != len(indices):
        raise ValueError("Length of obs must match length of indices")

    valid = set("XxYyZz")
    if any(c not in valid for c in obs):
        raise ValueError(f"Invalid character in obs. Only 'X, Y, Z' are allowed.")

    # Apply Pauli ops in sequence
    st = state
    for op, idx in zip(obs, indices):
        if op in "Xx":
            st = pauli_x(idx, st)
        elif op in "Yy":
            st = pauli_y(idx, st)
        elif op in "Zz":
            st = pauli_z(idx, st)

    # Compute ⟨ψ|st⟩ = measurement(state, ..., st)
    overlap = measurement(state, list(range(len(state[0][1]))), st)
    result = sum(coeff for coeff, _ in overlap)
    return simplify(result)

def schmidt_decomposition(pure_state):
    """
    Computes the Schmidt decomposition of a bipartite pure state |ψ⟩ in C^d_A ⊗ C^d_B.

    Parameters:
    - pure_state: list of (coefficient, (i, j)) tuples for basis |i⟩⊗|j⟩

    Returns:
    - List of symbolic Schmidt coefficients λ_i (i.e., square roots of singular values)
    """
    if not pure_state:
        return []

    # Infer subsystem dimensions from basis
    dim_A = max(i for _, (i, _) in pure_state) + 1
    dim_B = max(j for _, (_, j) in pure_state) + 1

    # Create symbolic matrix representing the state
    psi = Matrix.zeros(dim_A, dim_B)
    for coeff, (i, j) in pure_state:
        psi[i, j] += coeff

    # Perform symbolic SVD
    U, S, V = psi.singular_value_decomposition()

    # Extract non-zero Schmidt coefficients
    schmidt_coeffs = [val for val in S.diagonal() if val != 0]

    return schmidt_coeffs

In [13]:
ket0 = create_zero_state(1)
ket1 = pauli_x(0, ket0)

# Main codes

In [ ]:
'''
Note: Symbols (e.g., a, F) must appear only in the probabilities of mixed states.
Pure states (kets) must use numeric coefficients and bases to ensure stable behavior.
'''
# Set True if symbolic operations are required.
SYMBOLIC_MODE = True
# SYMBOLIC_MODE = False

In [11]:
print(schmidt_decomposition(create_bell_state(2, 0, 0)))

[sqrt(2)/2, sqrt(2)/2]


## Quantum Teleportation

In [ ]:
# Step 1: Prepare the input state |ψ⟩ = a|0⟩ + b|1⟩
psi = [(Rational(3,5), (0,)), (Rational(4,5), (1,))]  # Adjust coefficients
display_state(psi, "Input state")

# Step 2: Create a Bell pair between qubit 1 and 2: (|00⟩ + |11⟩)/√2
bell = create_bell_state(2, 0, 0)  # qubits 1 and 2

# Step 3: Combine all three qubits into one system: psi ⊗ bell
state = comp_st(psi, bell)  # qubits 0, 1, 2
display_state(state, "Initial state")

# Step 4: Alice applies CNOT(0,1) and H(0)
state = cnot(0, 1, state)
state = hadamard(0, state)

# Step 5: Prepare measurement outcomes in the computational basis
# There are four possible outcomes: 00, 01, 10, 11 on qubits 0 and 1
meas_basis = {
    (0, 0): [(1, (0, 0))],
    (0, 1): [(1, (0, 1))],
    (1, 0): [(1, (1, 0))],
    (1, 1): [(1, (1, 1))]
}

for outcome, meas in meas_basis.items():
    # Step 6: Alice measures her systems, then informs the outcomes to Bob.
    measured = measurement(meas, [0, 1], state)
    if not measured:
        continue
    prob, after_meas = normalize(measured)
    print()
    print(f"Measurement outcome: {outcome}")
    print(f"Probability: {prob}")
    display_state(after_meas, "After measurement")

    # Step 7: Bob applies correction depending on Alice's outcome
    i, j = outcome
    corrected = after_meas
    if i == 1:
        corrected = pauli_z(0, corrected)
    if j == 1:
        corrected = pauli_x(0, corrected)
    display_state(corrected, "Final state")

Input state: [(3/5, (0,)), (4/5, (1,))]
Initial state: [(3*sqrt(2)/10, (0, 0, 0)), (3*sqrt(2)/10, (0, 1, 1)), (2*sqrt(2)/5, (1, 0, 0)), (2*sqrt(2)/5, (1, 1, 1))]

Measurement outcome: (0, 0)
Probability: 1/4
After measurement: [(3/5, (0,)), (4/5, (1,))]
Final state: [(3/5, (0,)), (4/5, (1,))]

Measurement outcome: (0, 1)
Probability: 1/4
After measurement: [(4/5, (0,)), (3/5, (1,))]
Final state: [(3/5, (0,)), (4/5, (1,))]

Measurement outcome: (1, 0)
Probability: 1/4
After measurement: [(3/5, (0,)), (-4/5, (1,))]
Final state: [(3/5, (0,)), (4/5, (1,))]

Measurement outcome: (1, 1)
Probability: 1/4
After measurement: [(4/5, (0,)), (-3/5, (1,))]
Final state: [(3/5, (0,)), (4/5, (1,))]


## Detusch Algorithm

In [ ]:
# Step 1: Initial state |0⟩⊗|1⟩
state = create_zero_state(2) # |0⟩⊗|0⟩
state = pauli_x(1,state) # |0⟩⊗|1⟩

# Step 2: Apply Hadamard to both qubits
state = hadamard(0, state)
state = hadamard(1, state)

# Step 3: Apply Oracle U_f

# Constant functions
# state = id(state)
# state = pauli_x(1, state)

# Balanced functions
state = cnot(0, 1, state)
# state = pauli_x(1, cnot(0, 1, state))

# Step 4: Hadamard on qubit 0
state = hadamard(0, state)
display_state(state, "State before measurement")

# Step 5: Trace out the qubit 1
prob0, _ = normalize(measurement(ket0,[0],state))
prob1, _ = normalize(measurement(ket1,[0],state))

# state = pure_to_mixed(state)
# meas0 = pure_to_mixed(ket0)
# meas1 = pure_to_mixed(ket1)
# prob0 = prob_sum(mixed_measurement(meas0,[0],state))
# prob1 = prob_sum(mixed_measurement(meas1,[0],state))

print(f"Probability of measuring 0: {prob0}")
print(f"Probability of measuring 1: {prob1}")

State before measurement: [(sqrt(2)/2, (1, 0)), (-sqrt(2)/2, (1, 1))]
Probability of measuring 0: 0
Probability of measuring 1: 1


## BBPSSW entanglement purification protocol

In [14]:
# Enable symbolic mode for exact algebraic manipulation
SYMBOLIC_MODE = True

# Define symbolic fidelity parameter (must be in [0, 1])
F = symbols('F', positive=True)

# Step 1: Construct the initial isotropic state
# The state is a mixture of all four Bell states, weighted by fidelity F
isotropic_state = []
isotropic_state.append((F, create_bell_state(2, 0, 0)))           # Target Bell state |Φ⁺⟩
isotropic_state.append(((1 - F)/3, create_bell_state(2, 0, 1)))    # |Φ⁻⟩
isotropic_state.append(((1 - F)/3, create_bell_state(2, 1, 0)))    # |Ψ⁺⟩
isotropic_state.append(((1 - F)/3, create_bell_state(2, 1, 1)))    # |Ψ⁻⟩

# Display the initial noisy state
display_mixed_state(isotropic_state, "Isotropic state")

# Step 2: Create a two-copy state ρ ⊗ ρ
state = comp_mixed_states(isotropic_state, isotropic_state)  # Qubits 0-1 and 2-3

# Step 3: Apply bilateral CNOT gates
# Alice and Bob each apply CNOT between their two qubits
# CNOT(0→2), CNOT(1→3)
state = apply_gate_to_mixed_state(cnot, (0, 2), state)
state = apply_gate_to_mixed_state(cnot, (1, 3), state)

# Step 4: Define post-selection measurement (keep only |00⟩ and |11⟩ outcomes on qubits 2, 3)
meas00 = pure_to_mixed(comp_st(ket0, ket0))  # |00⟩
meas11 = pure_to_mixed(comp_st(ket1, ket1))  # |11⟩
success_meas = add_operators(meas00, meas11)     # Projector onto {|00⟩, |11⟩}

# Display the measurement used for post-selection
display_mixed_state(success_meas, "Success measurements")

# Step 5: Apply the measurement to the state
# Measure qubits 2 and 3 (auxiliary copy), trace them out after post-selection
success_state = mixed_measurement(success_meas, [2, 3], state)
display_mixed_state(success_state, "Unnormalized state after success")

# Step 6: Compute success probability (trace of the resulting state)
success_prob, norm_state = normalized_mixed_state(success_state)
print(f"Success probability: {success_prob}")

# Step 7: Compute resulting fidelity with respect to target Bell state |Φ⁺⟩
# success_fidelity = fidelity(create_bell_state(2, 0, 0), success_state) / success_prob
success_fidelity = fidelity(create_bell_state(2, 0, 0), norm_state)
print(f"Success fidelity: {success_fidelity}")

Isotropic state:
  [0] Prob: F, State: [(sqrt(2)/2, (0, 0)), (sqrt(2)/2, (1, 1))]
  [1] Prob: 1/3 - F/3, State: [(sqrt(2)/2, (0, 0)), (-sqrt(2)/2, (1, 1))]
  [2] Prob: 1/3 - F/3, State: [(sqrt(2)/2, (0, 1)), (sqrt(2)/2, (1, 0))]
  [3] Prob: 1/3 - F/3, State: [(sqrt(2)/2, (0, 1)), (-sqrt(2)/2, (1, 0))]
Success measurements:
  [0] Prob: 1, State: [(1, (0, 0))]
  [1] Prob: 1, State: [(1, (1, 1))]
Unnormalized state after success:
  [0] Prob: F**2 + (1/3 - F/3)**2, State: [(sqrt(2)/2, (0, 0)), (sqrt(2)/2, (1, 1))]
  [1] Prob: 2*F*(1/3 - F/3), State: [(sqrt(2)/2, (0, 0)), (-sqrt(2)/2, (1, 1))]
  [2] Prob: 2*(1/3 - F/3)**2, State: [(sqrt(2)/2, (0, 1)), (sqrt(2)/2, (1, 0))]
  [3] Prob: 2*(1/3 - F/3)**2, State: [(sqrt(2)/2, (0, 1)), (-sqrt(2)/2, (1, 0))]
Success probability: F**2 + 2*F*(1/3 - F/3) + 5*(1/3 - F/3)**2
Success fidelity: (F**2 + (1/3 - F/3)**2)/(F**2 + 2*F*(1/3 - F/3) + 5*(1/3 - F/3)**2)


In [15]:
mixed_to_DM(norm_state)

Matrix([
[ (4*F**2 + 4*F + 1)/(2*(8*F**2 - 4*F + 5)),                                     0,                                     0, (16*F**2 - 8*F + 1)/(2*(8*F**2 - 4*F + 5))],
[                                         0, 2*(F**2 - 2*F + 1)/(8*F**2 - 4*F + 5),                                     0,                                          0],
[                                         0,                                     0, 2*(F**2 - 2*F + 1)/(8*F**2 - 4*F + 5),                                          0],
[(16*F**2 - 8*F + 1)/(2*(8*F**2 - 4*F + 5)),                                     0,                                     0,  (4*F**2 + 4*F + 1)/(2*(8*F**2 - 4*F + 5))]])